In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("overview-of-recordings.csv")

In [ ]:
# Display the first few rows of the DataFrame
print(df.head())

# Check the distribution of phrases and prompts
print("Unique phrases:", df['phrase'].nunique())
print("Unique prompts:", df['prompt'].nunique())

   audio_clipping  audio_clipping:confidence background_noise_audible  \
0     no_clipping                     1.0000              light_noise   
1  light_clipping                     0.6803                 no_noise   
2     no_clipping                     1.0000                 no_noise   
3     no_clipping                     1.0000              light_noise   
4     no_clipping                     1.0000                 no_noise   

   background_noise_audible:confidence  overall_quality_of_the_audio  \
0                               1.0000                          3.33   
1                               0.6803                          3.33   
2                               0.6655                          3.33   
3                               1.0000                          3.33   
4                               1.0000                          4.67   

     quiet_speaker  quiet_speaker:confidence  speaker_id  \
0  audible_speaker                       1.0    43453425   
1  audib

In [ ]:
pip install nltk


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import nltk
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Function to preprocess text
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize text
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

# Apply preprocessing to 'phrase' and 'prompt' columns
df['clean_phrase'] = df['phrase'].apply(preprocess_text)
df['clean_prompt'] = df['prompt'].apply(preprocess_text)

In [ ]:
df_text = df[['phrase', 'prompt']]
df_text

,phrase,prompt
0,When I remember her I feel down,Emotional pain
1,When I carry heavy things I feel like breaking...,Hair falling out
2,there is too much pain when i move my arm,Heart hurts
3,My son had his lip pierced and it is swollen a...,Infected wound
4,My muscles in my lower back are aching,Infected wound
...,...,...
6656,I feel a burning sensation in my guts about 2 ...,Stomach ache
6657,I have a split on my thumb that will not heal.,Open wound
6658,I feel a lot of pain in the joints.,Joint pain
6659,The area around my heart doesn't feel good.,Heart hurts


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split

# Example dataset (replace it with your actual dataset)
phrases = df['phrase'].tolist()
prompts = df['prompt'].tolist()

# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(phrases)
X = tokenizer.texts_to_sequences(phrases)

# Padding sequences
max_length = max([len(seq) for seq in X])
X_padded = pad_sequences(X, maxlen=max_length, padding='post')

# Convert labels to one-hot encoding
label_to_index = {label: i for i, label in enumerate(set(prompts))}
y = [label_to_index[label] for label in prompts]
y_one_hot = np.zeros((len(y), len(label_to_index)))
for i, label_index in enumerate(y):
    y_one_hot[i, label_index] = 1

# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_one_hot, test_size=0.2, random_state=42)

# Define model architecture
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(LSTM(units=128))
model.add(Dense(units=len(label_to_index), activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/10


I0000 00:00:1708331269.262323     100 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


134/134 [==============================] - 13s 77ms/step - loss: 3.0687 - accuracy: 0.0774 - val_loss: 2.4544 - val_accuracy: 0.2083
Epoch 2/10
134/134 [==============================] - 2s 16ms/step - loss: 1.9177 - accuracy: 0.3151 - val_loss: 1.5308 - val_accuracy: 0.4174
Epoch 3/10
134/134 [==============================] - 1s 10ms/step - loss: 1.2370 - accuracy: 0.5209 - val_loss: 1.1429 - val_accuracy: 0.5910
Epoch 4/10
134/134 [==============================] - 1s 10ms/step - loss: 0.8010 - accuracy: 0.7037 - val_loss: 0.7875 - val_accuracy: 0.6923
Epoch 5/10
134/134 [==============================] - 1s 10ms/step - loss: 0.5088 - accuracy: 0.8351 - val_loss: 0.5332 - val_accuracy: 0.8321
Epoch 6/10
134/134 [==============================] - 1s 8ms/step - loss: 0.4582 - accuracy: 0.8618 - val_loss: 0.3263 - val_accuracy: 0.9268
Epoch 7/10
134/134 [==============================] - 1s 9ms/step - loss: 0.1663 - accuracy: 0.9629 - val_loss: 0.1354 - val_accuracy: 0.9756
Epoch 8/10
